# Data Algorithms

## 1. K-Means Clustering

### 1.1 Description

As stated on Wikipedia
> Given a set of observations $(\mathbf{x}_1, \mathbf{x}_2, \ldots, \mathbf{x}_n)$, where each observation is a *d*-dimensional real vector, $k$-means clustering aims to partition the $n$ observations into $k$ ($\leq n$) sets $\mathbf{S}= \{S_1, S_2, \ldots, S_k\}$ so as to minimize the within-cluster sum of squares (WCSS) (sum of distance functions of each point in the cluster to the K center). In other words, its objective is to find:
>
>$\begin{equation}\underset{\mathbf{S}} {\operatorname{arg\,min}}  \sum_{i=1}^{k} \sum_{\mathbf{x} \in S_i} \left\| \mathbf x - \boldsymbol\mu_i \right\|^2\end{equation}$
>
>where $\boldsymbol\mu_i$ is the mean of points in $S_i$.


### 1.2 Algorithm

1. Initialize cluster centroids $\boldsymbol\mu_1, \boldsymbol\mu_2, \ldots, \boldsymbol\mu_k \in \mathbb{R}^n$ 
2. Repeat until convergence:  
  2.1 For every $i$, set $y_i = \arg\min \|\mathbf{x}_i - \boldsymbol\mu_j\|^2$  
  2.2 For each $j$, set $\boldsymbol\mu_j = \frac{\sum_i^m1\{y_i = j\}\mathbf{x}_i}{}$

### 1.3 Objective

Write your own version of k-means algorithm using [Spark's RDD API](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD)

### 1.4 Dataset

In [ ]:
import numpy
from sklearn.datasets import make_blobs

In [ ]:
N_SAMPLES = 10**3
N_FEATURES= 2
N_CENTERS = 4
STD = 0.5
BOX = (-10.0, 10.0)
features, tlabels = make_blobs(n_samples=N_SAMPLES, 
                               n_features=N_FEATURES, 
                               centers=N_CENTERS, 
                               cluster_std=STD, 
                               center_box=BOX)

rdd = sc.parallelize(features).cache()

### 1.5 Implementation

In [ ]:
def compute_label(point, centroids):
    """Return the label of the closest centroid.
    """
    return <FILL IN>

def kmeans(rdd, n_centers, n_iter)
    # 0. Compute dataset bounding box
    # 1. Initialize cluster centroids
    # 2. Repeat until convergence
    #   2.1 Compute label for each points
    #   2.2 Update centroids
    <FILL IN>


In [ ]:
def compute_label(point, centroids):
    return numpy.argmin([numpy.linalg.norm(point - centroid) for centroid in centroids])

def kmeans(rdd, n_centers, n_iter):
    centroids = numpy.random.uniform(low=BOX[0], high=BOX[1], size=(n_centers, N_FEATURES))
    for i in range(n_iter):
        labels = rdd.map(lambda x: compute_label(x, centroids))

        centroid_map = labels.zip(rdd)\
                             .mapValues(lambda x: (x, 1))\
                             .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))\
                             .mapValues(lambda x: x[0] / x[1])\
                             .collectAsMap()
        for i in range(n_centers):
            centroids[i] = centroid_map.get(i, numpy.random.uniform(low=BOX[0], high=BOX[1], size=(1, N_FEATURES)))
    return centroids

kmeans(rdd, N_CENTERS, 10)

### 1.6 Compare with MLlib Implementation

In [ ]:
from pyspark.mllib.clustering import KMeans, KMeansModel

clusters = KMeans.train(rdd, N_CENTERS, maxIterations=10, runs=1, initializationMode="random")
clusters.centers